<a href="https://colab.research.google.com/github/khushnaidu/MyProjects/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras import layers
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import cifar10
from keras.utils import to_categorical

def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu', batch_normalization=True, conv_first=True):
    conv = layers.Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer='he_normal')

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
    else:
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v1(input_shape, depth, num_classes=10):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, in [a])')
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = layers.Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)

    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:
                strides = 2  # downsample
            y = resnet_layer(inputs=x, num_filters=num_filters, strides=strides)
            y = resnet_layer(inputs=y, num_filters=num_filters, activation=None)

            if stack > 0 and res_block == 0:
                x = resnet_layer(inputs=x, num_filters=num_filters, kernel_size=1, strides=strides, activation=None, batch_normalization=False)
            x = layers.add([x, y])
            x = layers.Activation('relu')(x)

        num_filters *= 2

    x = layers.AveragePooling2D(pool_size=8)(x)
    y = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(y)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define ResNet20 and ResNet32 models
model_resnet20 = resnet_v1(input_shape=(32, 32, 3), depth=20)
model_resnet32 = resnet_v1(input_shape=(32, 32, 3), depth=32)

# Compile models
model_resnet20.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet32.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train ResNet20
history_resnet20 = model_resnet20.fit(x_train, y_train, validation_split=0.1, epochs=50, batch_size=128, verbose=2)

# Train ResNet32
history_resnet32 = model_resnet32.fit(x_train, y_train, validation_split=0.1, epochs=50, batch_size=128, verbose=2)

# Evaluate on test set
loss_resnet20, accuracy_resnet20 = model_resnet20.evaluate(x_test, y_test, verbose=0)
loss_resnet32, accuracy_resnet32 = model_resnet32.evaluate(x_test, y_test, verbose=0)

print(f"ResNet20 - Test Loss: {loss_resnet20:.4f}, Test Accuracy: {accuracy_resnet20:.4f}")
print(f"ResNet32 - Test Loss: {loss_resnet32:.4f}, Test Accuracy: {accuracy_resnet32:.4f}")


170498071/170498071 [==============================] - 4s 0us/step
Epoch 1/50
352/352 - 32s - loss: 1.4648 - accuracy: 0.4700 - val_loss: 1.6474 - val_accuracy: 0.4404 - 32s/epoch - 91ms/step
Epoch 2/50
352/352 - 10s - loss: 1.0506 - accuracy: 0.6224 - val_loss: 1.5070 - val_accuracy: 0.5180 - 10s/epoch - 29ms/step
Epoch 3/50
352/352 - 10s - loss: 0.8582 - accuracy: 0.6949 - val_loss: 1.5414 - val_accuracy: 0.5584 - 10s/epoch - 29ms/step
Epoch 4/50
352/352 - 10s - loss: 0.7347 - accuracy: 0.7405 - val_loss: 0.9674 - val_accuracy: 0.6676 - 10s/epoch - 28ms/step
Epoch 5/50
352/352 - 10s - loss: 0.6331 - accuracy: 0.7788 - val_loss: 1.0121 - val_accuracy: 0.6584 - 10s/epoch - 29ms/step
Epoch 6/50
352/352 - 10s - loss: 0.5533 - accuracy: 0.8068 - val_loss: 1.0951 - val_accuracy: 0.6460 - 10s/epoch - 29ms/step
Epoch 7/50
352/352 - 10s - loss: 0.4899 - accuracy: 0.8283 - val_loss: 1.2836 - val_accuracy: 0.6270 - 10s/epoch - 30ms/step
Epoch 8/50
352/352 - 11s - loss: 0.4187 - accuracy: 0.8550